In [1]:
import os 
import sys
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
from catboost import CatBoostClassifier
sys.path.append('/Users/broniy/Desktop/creative')
%load_ext autoreload
%autoreload 2

In [2]:
clicked_df = pd.read_csv(
    "/Users/broniy/Desktop/creative/data/processed/clicked_variation_rows.csv"
)
non_clicked_df = pd.read_csv(
    "/Users/broniy/Desktop/creative/data/processed/non_clicked_variation_rows.csv"
)
creative_feats = pd.read_csv(
    "/Users/broniy/Desktop/creative/data/processed/combined_feats.csv"
).rename(columns={"experiment_id": "EXPERIMENT_ID", "variation_id": "VARIATION_ID"})

In [3]:
variations_df = (
    creative_feats
    .drop(columns=["Unnamed: 0"])
    .fillna(
        value={
            "Q1": "UNK",
            "Q2": "UNK",
            "Q3": "UNK",
            "Q4": "UNK",
            "Q5": "UNK",
            "Q6": "UNK",
            "Q7": "UNK",
            "Q8": "UNK",
            "Q9": "UNK",
            "Q10": "UNK",
            "category": "UNK",
        }
    )
)
variations_df

,VARIATION_ID,image_name,EXPERIMENT_ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,subject_line,category
0,b5759954-f5df-4934-961e-e4f99bc01bf4,82ad5eff-31db-4c81-b861-3f0b01294229.jpg,76a8268a-d19b-49ee-9c3d-69cf86f9f0f9,Model,Studio,Social Proof / Validation,Unknown,Bra & Underwear,Studio,One,26-35,Curvy,Brown,We take care of you ❤️,Emotional appeal
1,f13b8f3a-28a4-45f6-8f5c-5344412cb3a3,38d2823a-98c1-462d-b77d-e4ea85c899f2.jpg,76a8268a-d19b-49ee-9c3d-69cf86f9f0f9,Model,Lifestyle,Promotional / Incentive,Unknown,Bra & Underwear,Bedroom / Home,One,26-35,Curvy,Brown,asdfa,Conversational tone
2,eb261cf6-c634-41cc-968b-06df59c49599,a5ec01be-d9c3-4399-a846-f4edfd8868be.jpg,14990f86-a9df-4861-a08a-433e9242860d,Model,Studio,Social Proof / Validation,Unknown,Bra & Underwear,Studio,One,26-35,Curvy,Brown,We take care of you ❤️,Emotional appeal
3,a43a50b5-5177-456e-b2dd-ea45272e3cd2,210c5779-41cc-490c-b94f-2cffe2a48732.jpg,14990f86-a9df-4861-a08a-433e9242860d,Model,UGC,Social Proof / Validation,Unknown,Bra & Underwear,Bedroom / Home,One,26-35,Curvy,Black,Bra Shopping Nightmares? Not Anymore.,Problem–solution
4,2889f0ae-360b-4419-b304-49da3283b22e,fd656e58-25bf-4669-8d91-f20f2d3905da.jpg,14990f86-a9df-4861-a08a-433e9242860d,Model,Lifestyle,Social Proof / Validation,Unknown,Bra & Underwear,Natural Light / Lifestyle,One,26-35,Curvy,Black,Bra Shopping? We Give You 6 Months.,Signature repetition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,d33d584d-60a7-4b9d-8993-3d1b6786bc2d,6f142e2e-f2b5-4d7b-9f2d-ad017c619dd5.jpg,0ef6d2e9-7601-4df6-a215-83e6e79aa24e,Text,Unknown,Educational / Explanatory,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,UPDATE: Side Effects of Underoutfit ⚠️,Problem–solution
282,4eac4d70-7312-4a8a-8e8b-1f71747e0ec9,a5323a2e-b8a4-4218-861a-d82da1a35c1e.jpg,78a802ae-d6cd-4f39-aecb-138668fa2607,Model,Studio,Functional / Product-Focused,Shape & Silhouette,Bra,Studio,One,26-35,Slim,White,Every mom deserves this!,Emotional appeal
283,0dd3bab1-19bb-472b-93cd-038103dde79e,d409a3ce-1185-41f3-8718-154e3774512f.jpg,78a802ae-d6cd-4f39-aecb-138668fa2607,Model,Lifestyle,Functional / Product-Focused,Fit & Support,Bra,Everyday Lifestyle,One,26-35,Slim,White,"Real women, Real lift - Comfort you’ll love 💃",Sensory / lifestyle
284,637fdd90-26ed-42aa-affd-b27e5955b4a4,2fad7ea1-86ef-4c22-8b79-7bf9298f5de8.jpg,0ef6d2e9-7601-4df6-a215-83e6e79aa24e,Text,UGC,Educational / Explanatory,Unknown,Unknown,Unknown,Two,Unknown,Unknown,Unknown,UPDATE: underoutfit is contagious ⚠️,Conversational tone


In [4]:
users_df = (
    pd.concat([clicked_df, non_clicked_df], axis=0)
    .assign(
        CLICK=lambda x: x["CLICK_COUNT"] > 0,
        EXPERIMENT_DATE=lambda x: pd.to_datetime(
            {
                "year": 2025,
                "month": x["MONTH"],
                "day": x["DAY"],
            }
        ),
    )
    .dropna(subset=["CLICK_COUNT"])
    .drop(columns=["RN"])
    .fillna(
        value={
            "TOTAL_ORDERS_VALUE": 0,
            "AVG_ORDER_VALUE": 0,
            "LAST_ORDER_VALUE": 0,
            "COUNTRY": "UNK",
            "REGION": "UNK",
            "LATEST_CLICK_CLIENT_TYPE": "UNK",
            "LATEST_CLICK_CLIENT_NAME": "UNK",
            "LATEST_CLICK_CLIENT_OS_FAMILY": "UNK",
            "FIRST_UTM_SOURCE": "UNK",
            "FIRST_UTM_CONTENT": "UNK",
            "FIRST_UTM_CAMPAIGN": "UNK",
            "CITY": "UNK",
            "TIMEZONE": "UNK",
        }
    )
)
# Convert FIRST_ACTIVE_TS to datetime
users_df["FIRST_ACTIVE_TS_dt"] = pd.to_datetime(users_df["FIRST_ACTIVE_TS"])

# Compute months between today and FIRST_ACTIVE_TS
today = pd.Timestamp(datetime.today())

# Compute years and months difference and convert to total months
users_df["MONTHS_SINCE_FIRST_ACTIVE"] = (
    today.year - users_df["FIRST_ACTIVE_TS_dt"].dt.year
) * 12 + (today.month - users_df["FIRST_ACTIVE_TS_dt"].dt.month)

users_df = users_df[users_df["VARIATION_ID"].isin(variations_df["VARIATION_ID"])]
users_df

,EXPERIMENT_ID,VARIATION_ID,MONTH,DAY,HOUR,RECIPIENT_ID,CITY,COUNTRY,REGION,TIMEZONE,...,LATEST_CLICK_CLIENT_NAME,FIRST_ACTIVE_TS,FIRST_UTM_SOURCE,FIRST_UTM_CONTENT,FIRST_UTM_CAMPAIGN,CLICK_COUNT,CLICK,EXPERIMENT_DATE,FIRST_ACTIVE_TS_dt,MONTHS_SINCE_FIRST_ACTIVE
0,002deaf7-331f-4b5e-866b-f6dad60e4a79,148d62bc-044f-4c55-9195-8be2d7579d9d,7,28,12,01HNP4BCBMFV98J5B420PCS38F,UNK,United States,UNK,America/Chicago,...,Mobile Safari,2025-01-07 13:37:48.000,applovin,UNK,Underoutfit_CPP,2,True,2025-07-28,2025-01-07 13:37:48,9.0
1,e627d7f0-46c4-4894-872e-59a2fc108c30,56e5bc81-467f-4954-98ce-3c39a2f1c7e1,8,7,12,01HNP8Y1JFNW4G5YV6C5Z0X4SZ,Akron,United States,Ohio,America/New_York,...,Chrome Mobile,2025-07-06 13:31:02.000,Klaviyo,UNK,em - just dropped 460 459 457 - Sun Jul 6 20...,1,True,2025-08-07,2025-07-06 13:31:02,3.0
2,e4b4a349-3b14-439e-946f-f716101dac69,15be91b7-6b55-4cd3-8d07-f4068b018a3a,8,2,12,01HNP8Y1JFNW4G5YV6C5Z0X4SZ,Akron,United States,Ohio,America/New_York,...,Chrome Mobile,2025-07-06 13:31:02.000,Klaviyo,UNK,em - just dropped 460 459 457 - Sun Jul 6 20...,1,True,2025-08-02,2025-07-06 13:31:02,3.0
3,49c33d7c-ef04-43a7-bbd0-783489c64849,2dd79b7f-a399-459e-8eb0-6baf3a22f53b,9,6,13,01HNPADZ4935GQ7DA0Q76Z7J4V,Suffolk,United States,Virginia,America/New_York,...,Chrome,2025-06-30 15:31:02.000,Klaviyo,UNK,SMS Group B - Buy more save more - Mon Jun 30 ...,2,True,2025-09-06,2025-06-30 15:31:02,4.0
4,11c49e5d-21ac-4d6d-88c3-f211562a8e07,473c90cc-b034-4585-897c-da6900a9fa73,9,17,14,01HNPBGKJ35594G1B42RZ40RRV,Frontenac,United States,Missouri,America/Chicago,...,Mobile Safari,2025-09-18 03:20:58.000,UNK,UNK,UNK,1,True,2025-09-17,2025-09-18 03:20:58,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47695,8ea67496-0fb3-4efd-8cea-4b8d88351b8e,77a21e82-ef9f-458f-99ac-312f9a7a7a5c,7,1,12,01GC61RFSTG79C720FVYN9WQ3K,Marina Del Rey,United States,California,America/Los_Angeles,...,Mobile Safari,2025-07-28 21:35:29.000,Klaviyo,UNK,July’s Most Wanted Colors (EIKONA),0,False,2025-07-01,2025-07-28 21:35:29,3.0
47696,8ea67496-0fb3-4efd-8cea-4b8d88351b8e,77a21e82-ef9f-458f-99ac-312f9a7a7a5c,7,1,12,01GYZWHNDN8ZTKSZ96T349BAT5,Roanoke,United States,Virginia,America/New_York,...,Mobile Safari,2025-03-09 14:52:59.000,fb,120209600877760404,Shapewear CBO,0,False,2025-07-01,2025-03-09 14:52:59,7.0
47697,8ea67496-0fb3-4efd-8cea-4b8d88351b8e,77a21e82-ef9f-458f-99ac-312f9a7a7a5c,7,1,12,01GYZWHNDN8ZTKSZ96T349BAT5,Roanoke,United States,Virginia,America/New_York,...,Mobile Safari,2025-03-09 14:52:59.000,fb,120209600877760404,Shapewear CBO,0,False,2025-07-01,2025-03-09 14:52:59,7.0
47698,8ea67496-0fb3-4efd-8cea-4b8d88351b8e,77a21e82-ef9f-458f-99ac-312f9a7a7a5c,7,1,12,01JVPV1B7FE1SE4CA0FRYNQJ9H,Boydton,United States,Virginia,America/New_York,...,Chrome,2025-05-20 12:34:22.000,ig,120209362185440318,460 Neckline ASC+ 7DC1V Campaign,0,False,2025-07-01,2025-05-20 12:34:22,5.0


In [26]:
# Print the size of users_df before removal
print(f"users_df size before removing small experiments: {users_df.shape[0]} rows")
# Remove experiments with less than 100 participants
experiment_counts = users_df.groupby("EXPERIMENT_ID")["RECIPIENT_ID"].nunique()
valid_experiments = experiment_counts[experiment_counts >= 100].index
users_df = users_df[users_df["EXPERIMENT_ID"].isin(valid_experiments)]
# Print the size of users_df after removal
print(f"users_df size after removing small experiments: {users_df.shape[0]} rows")


users_df size before removing small experiments: 94756 rows
users_df size after removing small experiments: 94714 rows


In [27]:
users_df.sort_values("EXPERIMENT_DATE").groupby(["EXPERIMENT_ID", "EXPERIMENT_DATE"]).agg({"RECIPIENT_ID": "nunique"})

,,RECIPIENT_ID
EXPERIMENT_ID,EXPERIMENT_DATE,
002deaf7-331f-4b5e-866b-f6dad60e4a79,2025-07-28,2710
00bb26ff-6fe3-4465-ac77-12bfc33aa6df,2025-07-17,3574
0ef6d2e9-7601-4df6-a215-83e6e79aa24e,2025-10-06,2586
11c49e5d-21ac-4d6d-88c3-f211562a8e07,2025-09-17,2304
1d6dbba7-dcc5-46f4-a4aa-aef3124a8fcf,2025-09-05,3918
2a3f341e-1807-4eb3-9d8d-202c32d52632,2025-08-25,2150
3ecf34fc-1f15-4b32-970f-4061544da763,2025-07-14,3670
43d750b5-8698-4cf0-9ea2-f705f4f196ed,2025-09-25,3924
44d26695-cdf2-41a4-b161-393fdaf964bc,2025-07-26,4244


In [28]:
users_df.groupby(["EXPERIMENT_ID", "EXPERIMENT_DATE"]).agg({"CLICK": ["sum", "count"]})

CLICK      
                                                       sum count
EXPERIMENT_ID                        EXPERIMENT_DATE            
002deaf7-331f-4b5e-866b-f6dad60e4a79 2025-07-28       1355  2710
00bb26ff-6fe3-4465-ac77-12bfc33aa6df 2025-07-17       1787  3574
0ef6d2e9-7601-4df6-a215-83e6e79aa24e 2025-10-06       1293  2586
11c49e5d-21ac-4d6d-88c3-f211562a8e07 2025-09-17       1156  2312
1d6dbba7-dcc5-46f4-a4aa-aef3124a8fcf 2025-09-05       1959  3918
2a3f341e-1807-4eb3-9d8d-202c32d52632 2025-08-25       1075  2150
3ecf34fc-1f15-4b32-970f-4061544da763 2025-07-14       1835  3670
43d750b5-8698-4cf0-9ea2-f705f4f196ed 2025-09-25       1968  3936
44d26695-cdf2-41a4-b161-393fdaf964bc 2025-07-26       2122  4244
49c33d7c-ef04-43a7-bbd0-783489c64849 2025-09-06       1757  3514
6a258715-721a-41e9-8abb-af41308c1f48 2025-08-19       2051  4102
6f506df9-be60-452d-b914-8230c29c2ff1 2025-07-22       1622  3244
78a802ae-d6cd-4f39-aecb-138668fa2607 2025-10-02       1139  2278
81ae4870-e57d-4bc4-a2d7-48ffa5411707 2025-07-10       2113  4226
823158da-7b0a-4c19-8189-663c22a3ae38 2025-09-27       2286  4572
8ea67496-0fb3-4efd-8cea-4b8d88351b8e 2025-07-01       1766  3532
91eee220-fee7-488b-952a-c96aa8e493db 2025-08-14       1627  3254
9cd54b2b-31f9-43e4-9073-0d2b61bf9f15 2025-08-30       2039  4078
9defe9fd-0374-4de6-99f7-aaa392903d67 2025-08-12       2244  4488
a1db5d4b-b641-4d3d-a8a6-d4c9bdd9c033 2025-08-23       1543  3086
c5288ca2-3928-4364-8f08-bebc1036dd87 2025-07-11       2626  5252
cd4a656f-290a-41e2-be1d-bf62ad85757d 2025-09-29       2186  4372
e4b4a349-3b14-439e-946f-f716101dac69 2025-08-02        853  1706
e627d7f0-46c4-4894-872e-59a2fc108c30 2025-08-07        356   712
e697ab50-0abb-42d3-92a0-43f1ed597476 2025-08-29       3678  7356
f93bf2bd-1d50-4131-9ec2-223a4d9987e8 2025-09-23       2921  5842

In [30]:
USER_COLS = [
    "RECIPIENT_ID",
    "COUNTRY",
    "REGION",
    "LATEST_CLICK_CLIENT_TYPE",
    "LATEST_CLICK_CLIENT_NAME",
    "LATEST_CLICK_CLIENT_OS_FAMILY",
    "TOTAL_ORDERS_VALUE",
    "AVG_ORDER_VALUE",
    "LAST_ORDER_VALUE",
    "MONTHS_SINCE_FIRST_ACTIVE",
    "CLICK",
    "FIRST_UTM_SOURCE",
    "FIRST_UTM_CONTENT",
    "FIRST_UTM_CAMPAIGN",
    "CITY",
    "TIMEZONE"
]
VARIATION_COLS = [
    "Q1",
    "Q2",
    "Q3",
    "Q4",
    "Q5",
    "Q6",
    "Q7",
    "Q8",
    "Q9",
    "Q10",
    "category"
]

CATEGORICAL_COLS = [
    "COUNTRY",
    "REGION",
    "CITY",
    "TIMEZONE",
    "LATEST_CLICK_CLIENT_TYPE",
    "LATEST_CLICK_CLIENT_NAME",
    "LATEST_CLICK_CLIENT_OS_FAMILY",
    "FIRST_UTM_SOURCE",
    "FIRST_UTM_CONTENT",
    "FIRST_UTM_CAMPAIGN",
    "Q1",
    "Q2",
    "Q3",
    "Q4",
    "Q5",
    "Q6",
    "Q7",
    "Q8",
    "Q9",
    "Q10",
    "category"
]

NUMERICAL_COLS = [
    "TOTAL_ORDERS_VALUE",
    "AVG_ORDER_VALUE",
    "LAST_ORDER_VALUE",
    "MONTHS_SINCE_FIRST_ACTIVE"
]
COLS = CATEGORICAL_COLS + NUMERICAL_COLS

### Classification results

In [13]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

# Ensure experiment_date is datetime
users_df["EXPERIMENT_DATE"] = pd.to_datetime(users_df["EXPERIMENT_DATE"])

# Sort unique experiments by date
experiment_order = (
    users_df[["EXPERIMENT_ID", "EXPERIMENT_DATE"]]
    .sort_values("EXPERIMENT_DATE")
    .drop_duplicates()
    .reset_index(drop=True)
)
# Get last two for test, others for train
test_experiments = experiment_order.tail(2)["EXPERIMENT_ID"]
train_experiments = experiment_order.iloc[:-2]["EXPERIMENT_ID"]

# Join users_df with variation_df on EXPERIMENT_ID and VARIATION_ID
merged_df = users_df.merge(
    variations_df,
    left_on=["EXPERIMENT_ID", "VARIATION_ID"],
    right_on=["EXPERIMENT_ID", "VARIATION_ID"],
    how="left"
)

# Select rows for train/test
train_df = merged_df[merged_df["EXPERIMENT_ID"].isin(train_experiments)]
test_df= merged_df[merged_df["EXPERIMENT_ID"].isin(test_experiments)]
print(len(train_df), len(test_df))

display(merged_df.head())
X_train = train_df[ COLS]
y_train = train_df[ "CLICK"]
X_test = test_df[COLS]
y_test = test_df[ "CLICK"]

# Identify categorical features indices for CatBoost
cat_features = [X_train.columns.get_loc(col) for col in CATEGORICAL_COLS if col in X_train.columns]

clf = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    cat_features=cat_features,
    random_seed=42,
    verbose=100,
    # early_stopping_rounds=500,
    use_best_model=True
)
clf.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)

# If you want to report AUC on test:
from sklearn.metrics import roc_auc_score
y_pred = clf.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
print(f"Test AUC: {auc:.4f}")
print(f"Best validation score: {clf.best_score_['validation']['AUC']:.4f}")


89892 4864


,EXPERIMENT_ID,VARIATION_ID,MONTH,DAY,HOUR,RECIPIENT_ID,CITY,COUNTRY,REGION,TIMEZONE,...,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,subject_line,category
0,002deaf7-331f-4b5e-866b-f6dad60e4a79,148d62bc-044f-4c55-9195-8be2d7579d9d,7,28,12,01HNP4BCBMFV98J5B420PCS38F,UNK,United States,UNK,America/Chicago,...,Promotional / Incentive,Aesthetic Appeal,Bra,Studio,Two,26-35,Curvy,Unknown,These Colors Are July’s Hottest Picks,Sensory / lifestyle
1,e627d7f0-46c4-4894-872e-59a2fc108c30,56e5bc81-467f-4954-98ce-3c39a2f1c7e1,8,7,12,01HNP8Y1JFNW4G5YV6C5Z0X4SZ,Akron,United States,Ohio,America/New_York,...,Social Proof / Validation,Fit & Support,Bra & Underwear,Studio,One,26-35,Curvy,White,Fits like a dream,Sensory / lifestyle
2,e4b4a349-3b14-439e-946f-f716101dac69,15be91b7-6b55-4cd3-8d07-f4068b018a3a,8,2,12,01HNP8Y1JFNW4G5YV6C5Z0X4SZ,Akron,United States,Ohio,America/New_York,...,Social Proof / Validation,Unknown,Bra,Outdoor / Nature,One,26-35,Slim,White,Greatest (bra) of all time,Conversational tone
3,49c33d7c-ef04-43a7-bbd0-783489c64849,2dd79b7f-a399-459e-8eb0-6baf3a22f53b,9,6,13,01HNPADZ4935GQ7DA0Q76Z7J4V,Suffolk,United States,Virginia,America/New_York,...,Social Proof / Validation,Fit & Support,Bra,Bedroom / Home,One,18-25,Slim,White,Best Bra Ever!,Sensory / lifestyle
4,11c49e5d-21ac-4d6d-88c3-f211562a8e07,473c90cc-b034-4585-897c-da6900a9fa73,9,17,14,01HNPBGKJ35594G1B42RZ40RRV,Frontenac,United States,Missouri,America/Chicago,...,Emotional / Lifestyle,Seamlessness,Bra & Underwear,Studio,One,26-35,Slim,White,Just Restocked: Here Comes The Smooth 👰,Sensory / lifestyle


0:	test: 0.5422828	best: 0.5422828 (0)	total: 59.5ms	remaining: 59.4s
100:	test: 0.7987856	best: 0.8117623 (4)	total: 5.91s	remaining: 52.6s
200:	test: 0.8039856	best: 0.8117623 (4)	total: 11.8s	remaining: 46.8s
300:	test: 0.8040131	best: 0.8117623 (4)	total: 17.9s	remaining: 41.5s
400:	test: 0.8040317	best: 0.8117623 (4)	total: 23.8s	remaining: 35.6s
500:	test: 0.8051651	best: 0.8117623 (4)	total: 30.2s	remaining: 30s
600:	test: 0.8044604	best: 0.8117623 (4)	total: 36.3s	remaining: 24.1s
700:	test: 0.8044643	best: 0.8117623 (4)	total: 42.7s	remaining: 18.2s
800:	test: 0.8034017	best: 0.8117623 (4)	total: 49s	remaining: 12.2s
900:	test: 0.8022520	best: 0.8117623 (4)	total: 55s	remaining: 6.04s
999:	test: 0.8014596	best: 0.8117623 (4)	total: 1m	remaining: 0us

bestTest = 0.8117623369
bestIteration = 4

Shrink model to first 5 iterations.
Test AUC: 0.8118
Best validation score: 0.8118


### Train catboost via expanding window approach

In [32]:
def hit_rate_at_k(preds: pd.DataFrame, y_true: pd.DataFrame, k: int):
    top_k_preds = (
        preds
        .sort_values(
            ["EXPERIMENT_ID", "RECIPIENT_ID", "PRED"], ascending=[True, True, False]
        )
        .groupby(["EXPERIMENT_ID", "RECIPIENT_ID"])
        .head(k)
        
    )
    hit_rate = len(pd.merge(y_true, top_k_preds, on=["EXPERIMENT_ID", "RECIPIENT_ID", "VARIATION_ID"], how="inner"))/len(y_true)
    return hit_rate


In [34]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5, test_size=1)
idx = (
    users_df[["EXPERIMENT_ID", "EXPERIMENT_DATE"]]
    .sort_values("EXPERIMENT_DATE")
    .drop_duplicates()
    .reset_index(drop=True)
)
uplift_results = []
for train_idx, test_idx in tscv.split(idx):
    print(f"Train idx: {train_idx}, Test idx: {test_idx}")
    train_idx = idx.iloc[train_idx]["EXPERIMENT_ID"].values
    test_idx = idx.iloc[test_idx]["EXPERIMENT_ID"].values

    # prepare train data
    train_data = (
        users_df.set_index(["EXPERIMENT_ID", "VARIATION_ID"])
        .loc[train_idx, USER_COLS]
        .join(
            variations_df.set_index(["EXPERIMENT_ID", "VARIATION_ID"]).loc[
                train_idx, VARIATION_COLS
            ]
        )
    )
    train_X, train_y = train_data[COLS], train_data["CLICK"]

    # prepare test data
    test_data = (
        users_df.set_index(["EXPERIMENT_ID"])
        .loc[test_idx, USER_COLS]
        .join(
            variations_df.set_index(["EXPERIMENT_ID"]).loc[
                test_idx, ["VARIATION_ID"] + VARIATION_COLS
            ]
        )
    )
    test_X, test_y = (
        test_data[COLS],
        users_df.set_index(["EXPERIMENT_ID"]).loc[
            test_idx, ["CLICK", "RECIPIENT_ID", "VARIATION_ID"]
        ],
    )
    test_variations_per_experiment = len(
        users_df.set_index(["EXPERIMENT_ID"]).loc[test_idx]["VARIATION_ID"].unique()
    )
    print("Number of variations per experiment: ", test_variations_per_experiment)
    # Train CatBoost model
    print("Training CatBoost model...")
    model = CatBoostClassifier(
        iterations=500,
        learning_rate=0.03,
        depth=6,
        loss_function="Logloss",
        eval_metric="AUC",
        cat_features=cat_features,
        random_seed=42,
        verbose=100,
        # early_stopping_rounds=500,
    )
    model.fit(
        train_X.to_numpy(),
        train_y,
        cat_features=[
            train_X.columns.get_loc(col)
            for col in CATEGORICAL_COLS
            if col in train_X.columns
        ],
    )
    preds_ = model.predict_proba(test_X.to_numpy())[:, 1]
    preds = test_data.reset_index()[
        ["EXPERIMENT_ID", "RECIPIENT_ID", "VARIATION_ID"]
    ].assign(PRED=preds_)
    clicked_test_y = test_y[test_y["CLICK"] == True].reset_index()
    k = 1
    hit_rate = hit_rate_at_k(preds, clicked_test_y, k)
    base_hit_rate = 1 / test_variations_per_experiment
    uplift = (hit_rate - base_hit_rate) / base_hit_rate * 100
    print(f"Hit rate at k={k}: {hit_rate}, uplift: {uplift}%")
    uplift_results.append(uplift)
    # test_data = users_df.set_index(["EXPERIMENT_ID"]).loc[test_idx]
    # train_data = (users_df[users_df["RECIPIENT_ID"].isin(recipient_ids[train_idx])][USER_COLS]
    #     .join(variations_df[["EXPERIMENT_ID", "VARIATION_ID"]], on="EXPERIMENT_ID")
    # )
    # test_data = users_df.loc[recipient_ids[test_idx]]

    # train_X, train_y = train_data[cols], train_data["CLICK"]
    # test_X, test_y = test_data[cols], test_data["CLICK"]

Train idx: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20], Test idx: [21]
Number of variations per experiment:  5
Training CatBoost model...
0:	total: 51.1ms	remaining: 25.5s
100:	total: 4.88s	remaining: 19.3s
200:	total: 9.86s	remaining: 14.7s
300:	total: 15.1s	remaining: 9.99s
400:	total: 20.6s	remaining: 5.08s
499:	total: 25.9s	remaining: 0us
Hit rate at k=1: 0.21697154471544716, uplift: 8.485772357723576%
Train idx: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21], Test idx: [22]
Number of variations per experiment:  5
Training CatBoost model...
0:	total: 59.3ms	remaining: 29.6s
100:	total: 5.29s	remaining: 20.9s
200:	total: 10.9s	remaining: 16.1s
300:	total: 16.5s	remaining: 10.9s
400:	total: 22.1s	remaining: 5.46s
499:	total: 28.2s	remaining: 0us
Hit rate at k=1: 0.18547681539807523, uplift: -7.261592300962391%
Train idx: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22], Test idx: [23]
Number of variations per experiment:

In [35]:
uplift_results

[8.485772357723576,
 -7.261592300962391,
 -7.593778591033856,
 -2.721685689201059,
 7.965970610982209]